# Analyzing Galactic Rotation Curves Through the Application of the Holographic Principle
## Carson Downs
## August 2024

# Analyzing Galactic Rotation Curves

This project introduces a novel framework to analyze galactic rotation curves through the interaction of infotonic light with information structures at the event horizon of a universal black hole.

## Theoretical Background

The holographic principle suggests that information within a black hole is encoded on its two-dimensional surface, the event horizon. Infotonic light, which exists and travels in both our spatial dimensions and additional information dimensions (*w*-space), interacts with these information structures, creating mass and information density differentials that manifest as gravitational effects.

### Key Concepts

- **Infotonic Light**: A form of light acting as a bridge between spatial dimensions and w-space, influencing gravitational phenomena.
- **Event Horizon**: The surface where information is encoded, affecting the flow of infotonic light.
- **Information Density Differentials**: Variations in information density at the event horizon, observable as gravitational effects in galactic rotation curves.

This framework aims to provide a comprehensive method to predict galactic rotation curves, integrating concepts from quantum mechanics and string theory.

In [ ]:
# Load libraries
import requests
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from copy import deepcopy
from io import StringIO
from sklearn.metrics import mean_absolute_error, mean_squared_error

## Loading and Processing the SPARC Dataset

This section describes how the SPARC dataset, consisting of galactic rotation curves, is loaded from a GitHub repository and processed for analysis.

### Steps Involved

1. **Defining API URL**: The base URL for the GitHub API is set to access the repository containing .dat files with rotation curve data.

2. **Fetching File List**: A function `get_file_list(api_url)` retrieves the list of .dat files from the repository.

3. **Loading and Cleaning Data**: Each file is downloaded and processed with `load_and_clean(file_name, base_url)`, which:
   - Downloads the file from the GitHub repository.
   - Reads the data, skipping the first two rows of metadata.
   - Adds a column for the galaxy name.

4. **Combining Data**: Data from all files are combined into a single DataFrame `combined_data` with relevant columns renamed for clarity. The dataset contains an "extra" column due to a trailing tab character in the original .dat files' headers, which is dropped during cleaning.

5. **Displaying Data**: The combined dataset's shape and a preview of its contents are displayed. Data for a few selected galaxies are printed for verification.

6. **Exporting to CSV**: The processed data is saved as a CSV file for further use.

In [ ]:
# Define the base URL for the GitHub API
api_url = 'https://api.github.com/repos/carsondowns-cte/Rotmod_LTG/contents/'

# Function to get the list of files from the GitHub repository
def get_file_list(api_url):
    response = requests.get(api_url)
    if response.status_code == 200:
        file_list = [file['name'] for file in response.json() if file['name'].endswith('.dat')]
        return file_list
    else:
        print("Failed to retrieve file list from GitHub")
        return []

# Function to load and clean a file from the GitHub repository
def load_and_clean(file_name, base_url):
    url = base_url + file_name
    response = requests.get(url)
    if response.status_code == 200:
        data = StringIO(response.text)
        # Skip the first two rows of metadata and load the data
        df = pd.read_csv(data, delim_whitespace=True, skiprows=2)
        # Add a column for the galaxy name
        galaxy_name = file_name.replace('_rotmod.dat', '')
        df['galaxy'] = galaxy_name
        return df
    else:
        print(f"Failed to download {file_name}")
        return None

# Get the list of files from the GitHub repository
files = get_file_list(api_url)

# Define the base URL for the raw files
base_url = 'https://raw.githubusercontent.com/carsondowns-cte/Rotmod_LTG/main/'

# Initialize an empty list to store dataframes
dfs = []

# Load and clean each file
for file in files:
    df = load_and_clean(file, base_url)
    if df is not None:
        dfs.append(df)

# Combine all dataframes into a single dataframe
combined_data = pd.concat(dfs, ignore_index=True)

# Rename columns for clarity
combined_data.columns = ['radius', 'Vobs', 'errV', 'Vgas', 'Vdisk', 'Vbul', 'SBdisk', 'SBbul', 'extra', 'galaxy']

# Drop the extra column
combined_data = combined_data.drop(columns=['extra'])

# Display the shape and first few rows of the combined dataframe
print("Shape of combined dataframe:", combined_data.shape)
print(combined_data.head())

# Display data for 3 different galaxies
unique_galaxies = combined_data['galaxy'].unique()[:3]
for galaxy in unique_galaxies:
    galaxy_data = combined_data[combined_data['galaxy'] == galaxy]
    print(f"\nData for galaxy: {galaxy}")
    print(galaxy_data.head())

# Export the combined dataframe to CSV
combined_data.to_csv("sparc_dataset.csv", index=False)

## Optionally, the data can be loaded from the Rotmod_LTG.zip file included with the code as well. Uncomment the below code to do so.

In [ ]:
# import zipfile

# # Function to unzip the file and return the path to the extracted folder
# def unzip_file(zip_file_path):
#     extracted_folder = os.path.splitext(zip_file_path)[0]
#     with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
#         zip_ref.extractall(extracted_folder)
#     return extracted_folder

# # Function to load and clean a .dat file from the extracted folder
# def load_and_clean(file_path):
#     # Skip the first two rows of metadata and load the data
#     df = pd.read_csv(file_path, sep='\s+', skiprows=2)
#     # Add a column for the galaxy name
#     galaxy_name = os.path.basename(file_path).replace('_rotmod.dat', '')
#     df['galaxy'] = galaxy_name
#     return df

# # Specify the path to the zip file
# zip_file_path = 'Rotmod_LTG.zip'

# # Unzip the file and get the path to the extracted folder
# extracted_folder = unzip_file(zip_file_path)

# # Initialize an empty list to store dataframes
# dfs = []

# # Iterate through each .dat file in the extracted folder
# for root, dirs, files in os.walk(extracted_folder):
#     for file in files:
#         if file.endswith('.dat'):
#             file_path = os.path.join(root, file)
#             df = load_and_clean(file_path)
#             dfs.append(df)

# # Combine all dataframes into a single dataframe
# combined_data = pd.concat(dfs, ignore_index=True)

# # Rename columns for clarity
# combined_data.columns = ['radius', 'Vobs', 'errV', 'Vgas', 'Vdisk', 'Vbul', 'SBdisk', 'SBbul', 'extra', 'galaxy']

# # Drop the extra column
# combined_data = combined_data.drop(columns=['extra'])

# # Display the shape and first few rows of the combined dataframe
# print("Shape of combined dataframe:", combined_data.shape)
# print(combined_data.head())

# # Display data for 3 different galaxies
# unique_galaxies = combined_data['galaxy'].unique()[:3]
# for galaxy in unique_galaxies:
#     galaxy_data = combined_data[combined_data['galaxy'] == galaxy]
#     print(f"\nData for galaxy: {galaxy}")
#     print(galaxy_data.head())

# # Export the combined dataframe to CSV
# combined_data.to_csv("sparc_dataset.csv", index=False)

### Calculating Influence Components

This section defines functions to calculate the total influence of a galaxy, decompose it into baryonic and dark components, and compute the Nye factor. These calculations align with the theoretical framework described earlier.

In [ ]:
# Define the gravitational constant G
G = 6.67430e-11  # in m^3 kg^-1 s^-2

# Define function to calculate total influence
def calculate_total_influence(radius, Vobs):
    Vobs = Vobs * 10**3  # Convert velocity from km/s to m/s
    radius = radius * 3.086 * 10**19  # Convert radius from kpc to m
    sin_theta = Vobs**2 / radius**2  # Calculate sin(theta)
    total_influence = (radius**3 * sin_theta) / G  # Calculate total influence
    return total_influence

# Define function to calculate Nye factor and decompose total influence
def decompose_influence_components_infotonic_light(galaxy_data):
    # Calculate total influence for each row
    galaxy_data['Total_Influence'] = galaxy_data.apply(lambda row: calculate_total_influence(row['radius'], row['Vobs']), axis=1)

    # Calculate total influence of the galaxy
    total_influence_sum = galaxy_data['Total_Influence'].sum()

    # Calculate Nye factor using trigonometric identities
    galaxy_data['cos_theta'] = galaxy_data['Total_Influence'] / total_influence_sum
    galaxy_data['sin_theta'] = np.sqrt(1 - galaxy_data['cos_theta']**2)

    # Calculate baryonic and dark influence components
    galaxy_data['Baryonic_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['cos_theta']
    galaxy_data['Dark_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['sin_theta']

    total_baryonic_influence = galaxy_data['Baryonic_Influence'].sum()
    total_dark_influence = galaxy_data['Dark_Influence'].sum()
    nye_factor = galaxy_data['cos_theta'].mean()

    return total_baryonic_influence, total_dark_influence, nye_factor

# Initialize dictionary to store influence components for each galaxy
influence_components_infotonic_light = {}

# Apply the calculation to each galaxy
for galaxy in combined_data['galaxy'].unique():
    galaxy_data = combined_data[combined_data['galaxy'] == galaxy].copy()
    baryonic_influence, dark_influence, nye_factor = decompose_influence_components_infotonic_light(galaxy_data)
    influence_components_infotonic_light[galaxy] = {
        'Baryonic_Influence': baryonic_influence,
        'Dark_Influence': dark_influence,
        'Nye_Factor': nye_factor
    }

# Print influence components for each galaxy
print(f'Influence Components for each galaxy:')
for galaxy, values in influence_components_infotonic_light.items():
    print(f"Galaxy: {galaxy}, Baryonic Influence: {values['Baryonic_Influence']}, Dark Influence: {values['Dark_Influence']}, Nye Factor: {values['Nye_Factor']}")

### Creating a DataFrame from the Influence Components Dictionary

The following code converts the influence components dictionary into a DataFrame for easier analysis and inspection. The DataFrame is then saved to a CSV file for further use.

In [ ]:
# Checking data and converting the dictionary to a dataframe

# Create DataFrame directly from dictionary
influence_df = pd.DataFrame.from_dict(influence_components_infotonic_light, orient='index')

# Save DataFrame to CSV file
# influence_df.to_csv('influence_components_rotmod.csv')
influence_df.head()

### Adding Trigonometric Values and Verifying Pythagorean Identity

The following code adds columns for trigonometric values (`cos_theta` and `sin_theta`) derived from the Nye factor. It then verifies the Pythagorean identity (`cos²(theta) + sin²(theta) = 1`) to ensure the calculations are correct.

In [ ]:
# Add columns for trigonometric values
influence_df['cos_theta'] = influence_df['Nye_Factor']
influence_df['sin_theta'] = np.sqrt(1 - influence_df['cos_theta']**2)

# Verify the Pythagorean identity
influence_df['cos2_plus_sin2'] = influence_df['cos_theta']**2 + influence_df['sin_theta']**2

# Check if the values are approximately equal to 1
influence_df['is_identity_satisfied'] = np.isclose(influence_df['cos2_plus_sin2'], 1.0, atol=1e-6)

# Display the results
influence_df_summary = influence_df[['cos_theta', 'sin_theta', 'cos2_plus_sin2', 'is_identity_satisfied']]
print(influence_df_summary.head(10))

In [ ]:
# Export csv containing Nye Factors
influence_df.to_csv('influence_df.csv')

### Calculating and Decomposing Mass Components in Galaxies

This section outlines the process of calculating and decomposing the total mass of galaxies into baryonic and dark influence components, followed by the prediction of the galaxy's rotation curve. The key steps and calculations involved are as follows:

1. **Loading Data**:
   - The SPARC dataset is loaded from a CSV file containing galactic rotation curve data.

2. **Defining Constants and Functions**:
   - `kpc_to_m`: Conversion factor from kpc to meters.
   - `calculate_total_mass`: Converts radius and observed velocity to SI units and calculates total mass using the gravitational constant defined previously.

3. **Dynamic Nye Factor Calculation**:
   - The dynamic Nye factor is calculated based on the galaxy and radius, reflecting the relationship between the observed mass distribution and the baryonic component.

4. **Static Nye Factor Calculation**:
   - The static Nye factor is calculated for normalization. This factor represents the proportion of total mass at a specific radius relative to the sum of all total masses (`total_mass_sum`). This normalization ensures that the mass distribution is correctly proportioned across the galaxy.

5. **Decomposing Mass Influence Components**:
   - **Total Mass**: Calculated for each data point based on radius and observed velocity.
   - **Dynamic Nye Factor**: Applied to calculate baryonic influence.
   - **Static Nye Factor**: Used to compute dark influence.
   - **Percentages**: The percentages of baryonic and dark influences are calculated by dividing the respective influences by the total mass at each point and multiplying by 100.

6. **Normalization**:
   - The sum of baryonic and dark percentages is checked to ensure it sums to 100%. If not, adjustments are made to normalize these values.

7. **Predicting Rotation Curve**:
   - The predicted rotation curve is calculated by combining the baryonic and dark influence components, providing a predicted velocity for each radius.

8. **Clamping `cos_theta` Values**:
   - `cos_theta` values are clamped to the range [-1, 1] to prevent invalid square root operations in the calculation of `sin_theta`. This ensures numerical stability and avoids errors due to floating-point precision issues.

9. **Plotting Results**:
   - The observed and predicted rotation curves, along with the percentages of baryonic and dark influences, are plotted and saved for each galaxy.

This approach ensures a detailed and accurate analysis of the galactic rotation curves by decomposing the total mass into its fundamental components and predicting the rotational dynamics based on these influences.

### Application to Galactic Rotation Curves

The following process, derived from the paper, is implemented in the code below to analyze galactic rotation curves:

1. **Measure the Observed Rotational Velocity $V_{obs}$ and Radius \(r\)**:
   - Collect the observed data for the rotational velocity at various radii within the galaxy.

2. **Calculate the Gravitational Force**:
   $$
   F_{\text{gravity}} = \frac{V_{\text{obs}}^2}{r}
   $$

3. **Determine the Infotonic Light Flow**:
   - Calculate the equivalent force due to the differential pressure from the infotonic light flow:
   $$
   \sin(\theta) = \frac{F_{\text{gravity}}}{r} = \frac{V_{\text{obs}}^2}{r^2}
   $$

4. **Calculate the Equivalent Total Influence**:
   $$
   W_{\text{total}} = \frac{r^3 \cdot \sin(\theta)}{G} = \frac{r \cdot V_{\text{obs}}^2}{G}
   $$

5. **Decompose the Total Influence**:
   - Use the Nye Factor to separate the total influence into baryonic and dark components:
   $$
   W_{\text{baryonic}} = W_{\text{total}} \cdot N_{\text{dynamic}}
   $$
   $$
   W_{\text{dark}} = W_{\text{total}} \cdot \sqrt{1 - N_{\text{dynamic}}^2}
   $$

6. **Calculate the Velocity Contributions**:
   $$
   V_{\text{baryonic}} = \sqrt{\frac{G \cdot W_{\text{baryonic}}}{r}}
   $$
   $$
   V_{\text{dark}} = \sqrt{\frac{G \cdot W_{\text{dark}}}{r}}
   $$

7. **Calculate the Total Predicted Velocity**:
   $$
   V_{\text{pred}} = \sqrt{V_{\text{baryonic}}^2 + V_{\text{dark}}^2}
   $$

In [ ]:
# Load the CSV files
file_path = 'sparc_dataset.csv'
sparc_data = pd.read_csv(file_path)

# Constants
kpc_to_m = 3.086e+19  # Conversion factor from kpc to meters

def calculate_total_influence(radius, velocity):
    """
    Calculate the total mass at a given radius point based on the observed velocity.

    Parameters:
    radius (float): Radius in kiloparsecs (kpc).
    velocity (float): Observed velocity in kilometers per second (km/s).

    Returns:
    float: Total mass influence in solar masses (Msun).
    """
    radius_m = radius * kpc_to_m  # Convert radius from kpc to meters
    velocity_m_s = velocity * 1000  # Convert velocity from km/s to m/s
    total_influence = (radius_m * velocity_m_s**2) / G
    return total_influence

def calculate_dynamic_nye_factor(galaxy, radius):
    """
    Calculate the dynamic Nye factor for a given galaxy and radius.

    Parameters:
    galaxy (str): Name or identifier of the galaxy.
    radius (float): Radius in kiloparsecs (kpc).

    Returns:
    float: Dynamic Nye factor.
    """
    return influence_df.loc[galaxy, 'Nye_Factor'] / radius

def calculate_static_nye_factor(total_mass, total_mass_sum):
    """
    Calculate the static Nye factor for an individual point.

    Parameters:
    total_mass (float): Total mass at the current radius.
    total_mass_sum (float): Sum of all total masses.

    Returns:
    float: Static Nye factor for the current point.
    """
    return total_mass / total_mass_sum

def decompose_influence_components(galaxy, galaxy_data):
    """
    Decompose the total mass of a galaxy into baryonic and dark influence components.

    Parameters:
    galaxy (str): Name or identifier of the galaxy.
    galaxy_data (pd.DataFrame): DataFrame containing the galaxy data with columns 'radius' and 'Vobs'.

    Returns:
    pd.DataFrame: Updated DataFrame with columns for total mass, baryonic influence, dark influence,
                  and their respective percentages.
    """
    # Calculate total mass for each data point
    galaxy_data['Total_Influence'] = galaxy_data.apply(lambda row: calculate_total_influence(row['radius'], row['Vobs']), axis=1)

    # Calculate dynamic Nye factor for baryonic influence
    galaxy_data['Nye_Factor_Dynamic'] = galaxy_data['radius'].apply(lambda radius: calculate_dynamic_nye_factor(galaxy, radius))

    # Calculate baryonic influence using the dynamic Nye factor
    galaxy_data['Baryonic_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['Nye_Factor_Dynamic']

    # Calculate total mass sum (sum of all total masses for normalization)
    total_mass_sum = galaxy_data['Total_Influence'].sum()

    # Calculate static Nye factor for each data point
    galaxy_data['Nye_Factor_Static'] = galaxy_data['Total_Influence'].apply(lambda total_mass: calculate_static_nye_factor(total_mass, total_mass_sum))

    # Calculate dark influence using the static Nye factor
    galaxy_data['Dark_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['Nye_Factor_Static']

    # Calculate percentages of baryonic and dark influence
    galaxy_data['Percent_Baryonic'] = galaxy_data['Baryonic_Influence'] / galaxy_data['Total_Influence'] * 100
    galaxy_data['Percent_Dark'] = galaxy_data['Dark_Influence'] / galaxy_data['Total_Influence'] * 100

    # Check and normalize if necessary to ensure percentages sum to 100%
    total_percent = galaxy_data['Percent_Baryonic'] + galaxy_data['Percent_Dark']
    if not np.allclose(total_percent, 100, atol=1e-6):
        galaxy_data['Percent_Baryonic'] /= total_percent / 100
        galaxy_data['Percent_Dark'] /= total_percent / 100

    return galaxy_data

def calculate_predicted_rotation_curve(galaxy_data):
    """
    Calculate the predicted rotation curve for a galaxy based on its mass influence components.

    Parameters:
    galaxy_data (pd.DataFrame): DataFrame containing the galaxy data with columns 'radius',
                                'Total_Mass', and 'Nye_Factor_Dynamic'.

    Returns:
    tuple: A tuple containing:
        - r (np.ndarray): Array of radii in kiloparsecs (kpc).
        - Vpred (np.ndarray): Array of predicted velocities in kilometers per second (km/s).
    """
    r = galaxy_data['radius'].values * kpc_to_m  # Convert radius from kpc to meters
    total_mass = galaxy_data['Total_Influence'].values  # Mass in kg
    cos_theta = galaxy_data['Nye_Factor_Dynamic'].values

    # Clamp cos_theta values to be within the range [-1, 1]
    cos_theta = np.clip(cos_theta, -1, 1)

    # Calculate sin_theta ensuring the values are valid
    sin_theta = np.sqrt(1 - cos_theta**2)

    # Calculate the baryonic and dark influence components using the Nye factor
    V_baryonic = np.sqrt((G * total_mass * cos_theta) / r)
    V_dark = np.sqrt((G * total_mass * sin_theta) / r)

    # Combine the components to get the predicted velocity
    Vpred = np.sqrt(V_baryonic**2 + V_dark**2)

    return r / kpc_to_m, Vpred / 1000

# Create a directory to save the plots
plots_dir = 'galaxy_plots'
os.makedirs(plots_dir, exist_ok=True)

# Plot the observed vs. predicted rotation curves for each galaxy and save them
unique_galaxies = sparc_data['galaxy'].unique()

# Initialize an empty list to store the results for each galaxy
trig_results = []

# Plot the observed vs. predicted rotation curves for each galaxy and save them
unique_galaxies = sparc_data['galaxy'].unique()

for galaxy in unique_galaxies:
    if galaxy in influence_df.index:
        galaxy_data = sparc_data[sparc_data['galaxy'] == galaxy].copy()
        galaxy_data = decompose_influence_components(galaxy, galaxy_data)

        r, Vpred = calculate_predicted_rotation_curve(galaxy_data)

        fig, ax1 = plt.subplots()

        ax1.errorbar(galaxy_data['radius'], galaxy_data['Vobs'], yerr=galaxy_data['errV'], fmt='o', label='Observed')
        ax1.plot(r, Vpred, label='Predicted', linestyle='--')
        ax1.set_xlabel('Radius (kpc)')
        ax1.set_ylabel('Velocity (km/s)')
        ax1.set_title(f'Rotation Curve for Galaxy: {galaxy}')
        ax1.legend(loc='upper left')

        # Create a second y-axis for the percentage of baryonic and dark influence
        ax2 = ax1.twinx()
        ax2.plot(galaxy_data['radius'], galaxy_data['Percent_Baryonic'], label='Baryonic Influence Percentage', color='g', alpha=0.5)
        ax2.plot(galaxy_data['radius'], galaxy_data['Percent_Dark'], label='Dark Influence Percentage', color='purple', alpha=0.5)
        ax2.set_ylabel('Percentage')
        ax2.legend(loc='upper right')

        fig.tight_layout()  # Adjust layout to prevent overlap

        # Save the plot
        plot_filename = os.path.join(plots_dir, f'{galaxy}_rotation_curve.png')
        plt.savefig(plot_filename)

        plt.show()

        # Append the relevant data to results list
        trig_results.append({
            'galaxy': galaxy,
            'radius': galaxy_data['radius'].tolist(),
            'Vobs': galaxy_data['Vobs'].tolist(),
            'Vpred': Vpred.tolist(),
            'Percent_Baryonic': galaxy_data['Percent_Baryonic'].tolist(),
            'Percent_Dark': galaxy_data['Percent_Dark'].tolist()
        })

# Convert the results to a DataFrame and export to CSV
df_results = pd.DataFrame(trig_results)
df_results.to_csv('trigonometric_results.csv', index=False)

This code block creates a ZIP file containing all the plots generated and saved in the plots_dir directory.

In [ ]:
import zipfile

# Create a ZIP file with all the plots
zip_filename = 'galaxy_plots.zip'
with zipfile.ZipFile(zip_filename, 'w') as zipf:
    for root, dirs, files in os.walk(plots_dir):
        for file in files:
            zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), plots_dir))

print(f"All plots have been saved and zipped to {zip_filename}")

### Printing Galaxy Influence Distribution

This code defines a function to print the influence distribution for a given galaxy, including baryonic influence, dark influence, Nye factor, and their respective percentages of the total influence at a given radius. It formats the data into a table for easy visualization.

In [ ]:
def print_galaxy_mass_distribution(galaxy, galaxy_data):
    """
    Print the baryonic influence content, dark influence content, Nye factor, baryonic percentage,
    and dark percentage for a given galaxy as a table.

    Parameters:
    - galaxy: The name of the galaxy.
    - galaxy_data: DataFrame containing the mass influence distribution and Nye factor for the galaxy.
    """
    print(f"Mass Influence Distribution for Galaxy: {galaxy}")
    print(f"{'Radius (kpc)':>12} {'Baryonic Influence':>20} {'Dark Influence':>20} {'Nye Factor':>10} {'Baryonic Influence%':>12} {'Dark Influence %':>15}")
    print("-" * 90)

    for index, row in galaxy_data.iterrows():
        radius = row['radius']
        baryonic_mass = row['Baryonic_Influence']
        dark_influence = row['Dark_Influence']
        nye_factor = row['Nye_Factor_Dynamic']
        percent_baryonic = row['Percent_Baryonic']
        percent_dark = row['Percent_Dark']

        print(f"{radius:>12.2f} {baryonic_mass:>20.2e} {dark_influence:>20.2e} {nye_factor:>10.5f} {percent_baryonic:>12.2f} {percent_dark:>15.2f}")

# Example usage:
# Assuming `galaxy_data` contains the data for the galaxy 'CamB' after running the main script
galaxy = 'NGC1003'
galaxy_data = sparc_data[sparc_data['galaxy'] == galaxy].copy()
print(galaxy_data.head())
galaxy_data = decompose_influence_components(galaxy, galaxy_data)
print_galaxy_mass_distribution(galaxy, galaxy_data)

### Cross-Validation and Sensitivity Analysis

In addition to sensitivity analysis, cross-validation is employed to ensure the robustness and stability of the model. Despite the absence of a fitting function, cross-validation is valuable as it divides the data into training and validation sets multiple times, evaluating the consistency and reliability of the trigonometric calculations and mass decompositions across different subsets of data.

This combined approach provides a comprehensive understanding of the model's performance.

In [ ]:
from sklearn.model_selection import KFold

def cross_validate_model(galaxy, galaxy_data, k=5):
    """
    Perform cross-validation on the model to check for robustness.

    Parameters:
    - galaxy: The name of the galaxy.
    - galaxy_data: DataFrame containing the galaxy data.
    - k: Number of folds for cross-validation.

    Returns:
    - Average validation error.
    """
    kf = KFold(n_splits=k, shuffle=True, random_state=420)
    errors = []

    for train_index, test_index in kf.split(galaxy_data):
        train_data = galaxy_data.iloc[train_index].copy()
        test_data = galaxy_data.iloc[test_index].copy()

        # Fit the model on the training data
        train_data = decompose_influence_components(galaxy, train_data)

        # Predict on the test data
        test_data = decompose_influence_components(galaxy, test_data)
        r_test, Vpred_test = calculate_predicted_rotation_curve(test_data)

        # Calculate error (e.g., mean squared error)
        error = np.mean((test_data['Vobs'].values - Vpred_test)**2)
        errors.append(error)

    avg_error = np.mean(errors)
    print(f"Average validation error for galaxy {galaxy}: {avg_error}")
    return avg_error

def sensitivity_analysis(galaxy, galaxy_data, perturbation_factor=0.1):
    """
    Perform a sensitivity analysis on the model by perturbing the input data.

    Parameters:
    - galaxy: The name of the galaxy.
    - galaxy_data: DataFrame containing the galaxy data.
    - perturbation_factor: The factor by which to perturb the input data.

    Returns:
    - Perturbed predictions.
    """
    perturbed_data = galaxy_data.copy()
    # Perturb the observed velocities by the perturbation factor
    perturbed_data['Vobs'] *= (1 + perturbation_factor)

    # Debug: Print perturbed velocities
    print(f"Perturbed Velocities for {galaxy}:")
    print(perturbed_data[['radius', 'Vobs']])

    # Decompose the mass components using the perturbed data
    perturbed_data = decompose_influence_components(galaxy, perturbed_data)

    # Calculate the perturbed predicted rotation curve
    r, Vpred_perturbed = calculate_predicted_rotation_curve(perturbed_data)

    # Debug: Print perturbed predicted velocities
    print(f"Perturbed Predicted Velocities for {galaxy}:")
    print(list(zip(r, Vpred_perturbed)))

    # Print the perturbed data for comparison
    print("Perturbed Predictions:")
    print_galaxy_mass_distribution(galaxy, perturbed_data)

    # Plot the perturbed rotation curve
    fig, ax1 = plt.subplots()

    ax1.errorbar(perturbed_data['radius'], perturbed_data['Vobs'], yerr=perturbed_data['errV'], fmt='o', label='Observed')
    ax1.plot(r, Vpred_perturbed, label='Perturbed Predicted', linestyle='--')
    ax1.set_xlabel('Radius (kpc)')
    ax1.set_ylabel('Velocity (km/s)')
    ax1.set_title(f'Perturbed Rotation Curve for Galaxy: {galaxy}')
    ax1.legend(loc='upper left')

    # Create a second y-axis for the percentage of baryonic and dark matter
    ax2 = ax1.twinx()
    ax2.plot(perturbed_data['radius'], perturbed_data['Percent_Baryonic'], label='Baryonic Matter Percentage', color='g', alpha=0.5)
    ax2.plot(perturbed_data['radius'], perturbed_data['Percent_Dark'], label='Dark Matter Percentage', color='purple', alpha=0.5)
    ax2.set_ylabel('Percentage')
    ax2.legend(loc='upper right')

    fig.tight_layout()  # Adjust layout to prevent overlap

    # Save the plot
    plots_dir = 'galaxy_plots'
    os.makedirs(plots_dir, exist_ok=True)
    plot_filename = os.path.join(plots_dir, f'{galaxy}_perturbed_rotation_curve.png')
    plt.savefig(plot_filename)

    plt.show()

    # Plot sensitivity analysis results
    fig, ax = plt.subplots()
    ax.plot(r, np.abs(galaxy_data['Vobs'].values - Vpred_perturbed), label='Perturbation Error')
    ax.set_xlabel('Radius (kpc)')
    ax.set_ylabel('Error (km/s)')
    ax.set_title(f'Sensitivity Analysis for Galaxy: {galaxy}')
    ax.legend()

    plot_filename = os.path.join(plots_dir, f'{galaxy}_sensitivity_analysis.png')
    plt.savefig(plot_filename)

    plt.show()

# Example usage for galaxies 'UGC09992' and 'UGCA444'
galaxy1 = 'UGC09992'
galaxy_data1 = sparc_data[sparc_data['galaxy'] == galaxy].copy()

galaxy2 = 'UGCA444'
galaxy_data2 = sparc_data[sparc_data['galaxy'] == galaxy2].copy()

# Perform cross validation
cross_validate_model(galaxy1, galaxy_data1, k=5)
cross_validate_model(galaxy2, galaxy_data2, k=5)

# Perform sensitivity analysis with a perturbation factor
sensitivity_analysis(galaxy1, galaxy_data1, perturbation_factor=0.1)
sensitivity_analysis(galaxy2, galaxy_data2, perturbation_factor=0.1)

### Error Analysis for Different Perturbation Factors

This section performs a comprehensive error analysis to assess the robustness and accuracy of the model under various perturbation conditions. The Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) are calculated for normal and perturbed velocity predictions for each galaxy.

1. **Purpose**:
   - Evaluate model stability and sensitivity to changes in input data.
   - Determine the impact of perturbations on the model's predictive performance.

2. **Method**:
   - **Normal Predictions**: Errors calculated using original observed velocities.
   - **Perturbed Predictions**: Errors calculated after perturbing the observed velocities by a specified factor.

3. **Insights**:
   - Consistent MSE and RMSE values across normal predictions indicate the model’s reliability.
   - Increased MSE and RMSE in perturbed conditions reflect the model's sensitivity to input variations, highlighting areas for potential refinement.

By running this analysis, we gain valuable insights into the model’s performance across different scenarios, aiding in its further development and validation.

In [ ]:
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
import logging

logging.basicConfig(filename='galaxy_errors.log', level=logging.ERROR)

def calculate_errors(galaxy, galaxy_data, perturbation_factor=0.0, add_noise=False, random_seed=None, verbose=True):
    """
    Calculate MSE and RMSE for normal and perturbed results for a given galaxy.

    Parameters:
    - galaxy: The name of the galaxy.
    - galaxy_data: DataFrame containing the galaxy data.
    - perturbation_factor: The factor by which to perturb the input data.
    - add_noise: If True, adds small random noise to the observed velocities.
    - random_seed: Seed for the random number generator to ensure reproducibility.
    - verbose: If True, prints detailed errors for each galaxy.

    Returns:
    - Dictionary containing MSE and RMSE for normal and perturbed results.
    """
    try:
        # Optionally set the random seed for reproducibility
        if random_seed is not None:
            np.random.seed(random_seed)

        # Optionally add small random noise to observed velocities to test the model's sensitivity
        if add_noise:
            noise = np.random.normal(0, 0.001, len(galaxy_data))
            galaxy_data['Vobs'] += noise

        # Validate 'radius' column existence
        if 'radius' not in galaxy_data.columns:
            error_message = f"'radius' column missing in data for galaxy {galaxy}"
            logging.error(error_message)
            raise KeyError(error_message)
        if galaxy_data['radius'].isnull().any():
            error_message = f"'radius' column contains NaN values for galaxy {galaxy}"
            logging.error(error_message)
            raise ValueError(error_message)

        # Calculate normal predictions
        normal_data = decompose_influence_components(galaxy, galaxy_data.copy())
        r_normal, Vpred_normal = calculate_predicted_rotation_curve(normal_data)

        # Perturb the observed velocities if perturbation_factor is not zero
        if perturbation_factor != 0:
            perturbed_data = galaxy_data.copy()
            perturbed_data['Vobs'] *= (1 + perturbation_factor)
        else:
            perturbed_data = galaxy_data.copy()

        # Calculate perturbed predictions
        perturbed_data = decompose_influence_components(galaxy, perturbed_data)
        r_perturbed, Vpred_perturbed = calculate_predicted_rotation_curve(perturbed_data)

        # Handle NaN values in predictions
        Vpred_normal = np.nan_to_num(Vpred_normal, nan=np.nanmean(Vpred_normal))
        Vpred_perturbed = np.nan_to_num(Vpred_perturbed, nan=np.nanmean(Vpred_perturbed))

        # Calculate MSE and RMSE for normal predictions
        mse_normal = mean_squared_error(normal_data['Vobs'], Vpred_normal)
        rmse_normal = np.sqrt(mse_normal)

        # Calculate MSE and RMSE for perturbed predictions
        mse_perturbed = mean_squared_error(perturbed_data['Vobs'], Vpred_perturbed)
        rmse_perturbed = np.sqrt(mse_perturbed)

        if verbose:
            # Print results
            print(f"Errors for Galaxy: {galaxy}")
            print(f"{'':>20} {'MSE':>10} {'RMSE':>10}")
            print(f"{'Normal':>20} {mse_normal:>10.10f} {rmse_normal:>10.10f}")
            print(f"{'Perturbed':>20} {mse_perturbed:>10.10f} {rmse_perturbed:>10.10f}")

        # Return results as a dictionary
        return {
            'MSE_Normal': mse_normal,
            'RMSE_Normal': rmse_normal,
            'MSE_Perturbed': mse_perturbed,
            'RMSE_Perturbed': rmse_perturbed
        }
    except Exception as e:
        error_message = f"Error processing galaxy {galaxy}: {str(e)}"
        logging.error(error_message)
        raise ValueError(error_message)

def calculate_average_errors(sparc_data, perturbation_factors=[0], add_noise=False, random_seed=None, verbose=False):
    galaxies = sparc_data['galaxy'].unique()
    results = {pf: [] for pf in perturbation_factors}
    skipped_galaxies = []

    for galaxy in galaxies:
        galaxy_data = sparc_data[sparc_data['galaxy'] == galaxy].copy()
        for pf in perturbation_factors:
            try:
                errors = calculate_errors(galaxy, galaxy_data, perturbation_factor=pf, add_noise=add_noise, random_seed=random_seed, verbose=verbose)
                results[pf].append(errors)
            except Exception as e:
                print(f"Skipping galaxy {galaxy}: {str(e)}")
                skipped_galaxies.append((galaxy, str(e)))

    average_errors = {}
    for pf in perturbation_factors:
        df = pd.DataFrame(results[pf])
        average_errors[pf] = df.mean().to_dict()

    return average_errors, skipped_galaxies

# Load your SPARC data here
sparc_data = pd.read_csv('sparc_dataset.csv')

# Calculate and print the average errors for the perturbation factor 0 (no perturbation)
average_errors, skipped_galaxies = calculate_average_errors(sparc_data, perturbation_factors=[0], add_noise=False, random_seed=42, verbose=False)

print("Average Errors with No Perturbation:")
for pf, errors in average_errors.items():
    print(f"Perturbation Factor {pf}:")
    print(errors)

if skipped_galaxies:
    print("\nSkipped Galaxies:")
    for galaxy, reason in skipped_galaxies:
        print(f"Galaxy {galaxy} skipped due to: {reason}")

In [ ]:
# Extract relevant data for plotting best and worst performing galaxies
galaxies = sparc_data['galaxy'].unique()
performance_data = []

# Calculate errors for each galaxy and store the results
for galaxy in galaxies:
    galaxy_data = sparc_data[sparc_data['galaxy'] == galaxy].copy()
    try:
        errors = calculate_errors(galaxy, galaxy_data, perturbation_factor=0, verbose=False)
        performance_data.append((galaxy, errors['RMSE_Normal']))
    except Exception as e:
        print(f"Error processing galaxy {galaxy}: {str(e)}")

# Sort galaxies based on their RMSE_Normal
sorted_galaxies = sorted(performance_data, key=lambda x: x[1])

# Select three best and three worst performing galaxies
best_galaxies = sorted_galaxies[:3]
worst_galaxies = sorted_galaxies[-3:]

# Function to plot error metrics for the galaxies
def plot_error_metrics(galaxy_data, title):
    names = [g[0] for g in galaxy_data]
    metrics = [g[1] for g in galaxy_data]
    plt.figure(figsize=(10, 6))
    plt.bar(names, metrics, color='skyblue')
    plt.xlabel('Galaxy')
    plt.ylabel('RMSE')
    plt.title(title)
    plt.show()

# Plotting for the best and worst performing galaxies
plot_error_metrics(best_galaxies, 'Three Best Performing Galaxies')
plot_error_metrics(worst_galaxies, 'Three Worst Performing Galaxies')

# Display error metrics for the best and worst galaxies
print("Three Best Performing Galaxies:")
for galaxy, error in best_galaxies:
    print(f"Galaxy: {galaxy}, RMSE: {error}")

print("\nThree Worst Performing Galaxies:")
for galaxy, error in worst_galaxies:
    print(f"Galaxy: {galaxy}, RMSE: {error}")

## Solving for Further Variables

The additional insight of a constant $w_z$ component allows for the derication of several new variables to be used in the wave equation calculations:

1. **Effective Radius ($r_{\text{eff}}$)**:
   The effective radius combines the spatial radius and the temporal component to account for the effects in both dimensions:
   $$
   r_{\text{eff}} = \sqrt{r^2 + w_z^2}
   $$
   where $r$ represents the spatial radius and $w_z$ the temporal component.

2. **Total Influence ($W_{\text{total}}$)**:
   The total influence at a given radius is derived from the observed velocity, providing a measure of the combined mass effect at that radius:
   $$
   W_{\text{total}} = \frac{r \cdot V_{\text{obs}}^2}{G}
   $$
   This equation uses the observed rotational velocity $V_{\text{obs}}$ and the gravitational constant $G$ to calculate the total influence.

3. **Pressure Differentials ($\Delta P$)**:
   Pressure differentials are calculated to separate the contributions of baryonic and dark matter components:
   $$
   \Delta P_{\text{Baryonic}} = \frac{W_{\text{total}} \cdot r}{r_{\text{eff}}^2}
   $$
   $$
   \Delta P_{\text{Dark}} = \frac{W_{\text{total}} \cdot w_z}{r_{\text{eff}}^2}
   $$
   These equations relate the total influence to the effective radius, with $r$ and $w_z$ representing the spatial and temporal components, respectively.

4. **Wave Numbers ($k$)**:
   Wave numbers for baryonic and dark components are derived to describe the wave-like behavior of these influences:
   $$
   k_{\text{Baryonic}} = \frac{W_{\text{baryonic}}}{\Delta P_{\text{Baryonic}} \cdot r_{\text{eff}}}
   $$
   $$
   k_{\text{Dark}} = \frac{W_{\text{dark}}}{\Delta P_{\text{Dark}} \cdot r_{\text{eff}}}
   $$
   These expressions link the influence components with their corresponding pressure differentials and the effective radius.

5. **Potential Function Components**:
   The potential function $V(w)$ is decomposed into components related to baryonic and dark velocity contributions, reflecting their influence on the system:
   $$
   V_{\text{baryonic}} = \sqrt{\frac{G \cdot W_{\text{total}} \cdot W_{\text{wave}}}{r}}
   $$
   $$
   V_{\text{dark}} = \sqrt{\frac{G \cdot W_{\text{total}} \cdot (1 - W_{\text{wave}})}{r}}
   $$
   Here, $W_{\text{wave}}$ represents the informational influence, differentiating between baryonic and dark contributions.

6. **Predicted Velocity ($V_{\text{pred}}$)**:
   The predicted rotation curve is calculated by combining the baryonic and dark matter components using the wave function influence:
   $$
   V_{\text{pred}} = \sqrt{V_{\text{baryonic}}^2 + V_{\text{dark}}^2}
   $$
   The influence term here refers to the wave function solution's absolute value, which modulates the baryonic and dark matter contributions to the rotational velocity.

In [ ]:
# Constants
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
kpc_to_m = 3.086e+19  # Conversion factor from kpc to meters
WZ = 1.0  # Assuming a constant WZ length for simplicity

def calculate_total_influence(radius, velocity):
    """
    Calculate the total mass at a given radius point based on the observed velocity.

    Parameters:
    radius (float): Radius in kiloparsecs (kpc).
    velocity (float): Observed velocity in kilometers per second (km/s).

    Returns:
    float: Total mass influence in solar masses (Msun).
    """
    radius_m = radius * kpc_to_m  # Convert radius from kpc to meters
    velocity_m_s = velocity * 1000  # Convert velocity from km/s to m/s
    total_influence = (radius_m * velocity_m_s**2) / G
    return total_influence

def calculate_effective_radius(radius):
    """
    Calculate the effective radius given the spatial radius and WZ component.

    Parameters:
    radius (float): Spatial radius in kiloparsecs (kpc).

    Returns:
    float: Effective radius.
    """
    return np.sqrt(radius**2 + WZ**2)

def calculate_dynamic_nye_factor(galaxy, radius):
    """
    Calculate the dynamic Nye factor for a given galaxy and radius.

    Parameters:
    galaxy (str): Name or identifier of the galaxy.
    radius (float): Radius in kiloparsecs (kpc).

    Returns:
    float: Dynamic Nye factor.
    """
    return influence_df.loc[galaxy, 'Nye_Factor'] / radius

def calculate_static_nye_factor(total_mass, total_mass_sum):
    """
    Calculate the static Nye factor for an individual point.

    Parameters:
    total_mass (float): Total mass at the current radius.
    total_mass_sum (float): Sum of all total masses.

    Returns:
    float: Static Nye factor for the current point.
    """
    return total_mass / total_mass_sum

def decompose_influence_components(galaxy, galaxy_data):
    """
    Decompose the total mass of a galaxy into baryonic and dark influence components.

    Parameters:
    galaxy (str): Name or identifier of the galaxy.
    galaxy_data (pd.DataFrame): DataFrame containing the galaxy data with columns 'radius' and 'Vobs'.

    Returns:
    pd.DataFrame: Updated DataFrame with columns for total mass, baryonic influence, dark influence,
                  and their respective percentages.
    """
    # Calculate total mass for each data point
    galaxy_data['Total_Influence'] = galaxy_data.apply(lambda row: calculate_total_influence(row['radius'], row['Vobs']), axis=1)

    # Calculate effective radius for each data point
    galaxy_data['Effective_Radius'] = galaxy_data['radius'].apply(calculate_effective_radius)

    # Calculate dynamic Nye factor for baryonic influence
    galaxy_data['Nye_Factor_Dynamic'] = galaxy_data['radius'].apply(lambda radius: calculate_dynamic_nye_factor(galaxy, radius))

    # Calculate baryonic influence using the dynamic Nye factor
    galaxy_data['Baryonic_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['Nye_Factor_Dynamic']

    # Calculate total mass sum (sum of all total masses for normalization)
    total_mass_sum = galaxy_data['Total_Influence'].sum()

    # Calculate static Nye factor for each data point
    galaxy_data['Nye_Factor_Static'] = galaxy_data['Total_Influence'].apply(lambda total_mass: calculate_static_nye_factor(total_mass, total_mass_sum))

    # Calculate dark influence using the static Nye factor
    galaxy_data['Dark_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['Nye_Factor_Static']

    # Calculate pressure differentials
    galaxy_data['Delta_P_Baryonic'] = galaxy_data.apply(lambda row: (row['Total_Influence'] * row['radius']) / (row['Effective_Radius']**2), axis=1)
    galaxy_data['Delta_P_Dark'] = galaxy_data.apply(lambda row: (row['Total_Influence'] * WZ) / (row['Effective_Radius']**2), axis=1)

    # Solve for k
    galaxy_data['k_Baryonic'] = galaxy_data.apply(lambda row: row['Baryonic_Influence'] / (row['Delta_P_Baryonic'] * row['Effective_Radius']), axis=1)
    galaxy_data['k_Dark'] = galaxy_data.apply(lambda row: row['Dark_Influence'] / (row['Delta_P_Dark'] * row['Effective_Radius']), axis=1)

    # Calculate percentages of baryonic and dark influence
    galaxy_data['Percent_Baryonic'] = galaxy_data['Baryonic_Influence'] / galaxy_data['Total_Influence'] * 100
    galaxy_data['Percent_Dark'] = galaxy_data['Dark_Influence'] / galaxy_data['Total_Influence'] * 100

    # Check and normalize if necessary to ensure percentages sum to 100%
    total_percent = galaxy_data['Percent_Baryonic'] + galaxy_data['Percent_Dark']
    if not np.allclose(total_percent, 100, atol=1e-6):
        galaxy_data['Percent_Baryonic'] /= total_percent / 100
        galaxy_data['Percent_Dark'] /= total_percent / 100

    return galaxy_data

# Create a directory to save the plots
plots_dir = 'galaxy_plots_new'
os.makedirs(plots_dir, exist_ok=True)

# Initialize a DataFrame to store results for all galaxies
results_df = pd.DataFrame()

# Plot the observed vs. predicted rotation curves for each galaxy and save them
unique_galaxies = sparc_data['galaxy'].unique()

for galaxy in unique_galaxies:
    if galaxy in influence_df.index:
        galaxy_data = sparc_data[sparc_data['galaxy'] == galaxy].copy()
        galaxy_data = decompose_influence_components(galaxy, galaxy_data)

        r, Vpred = calculate_predicted_rotation_curve(galaxy_data)

        # Append additional data to the galaxy_data DataFrame
        galaxy_data['Predicted_Velocity'] = Vpred

        # Append galaxy data to the results_df DataFrame
        results_df = pd.concat([results_df, galaxy_data])

# Export the results DataFrame to a CSV file

results_csv_path = 'galaxy_influence_results.csv'
results_df.to_csv(results_csv_path, index=False)

results_df.head()

## Analyzing relationships between the newly derived fields

### Fields and Descriptions

- **Effective_Radius**
  - **Description**: The effective radius is the combined measure of the spatial radius and an additional component WZ. It represents the adjusted radius that takes into account extra-dimensional factors, providing a more comprehensive measure of the distance from the center of the galaxy.

- **Delta_P_Baryonic**
  - **Description**: The baryonic pressure differential ($\Delta P_{\text{baryonic}}$) indicates the pressure difference created by baryonic matter (ordinary matter) within the galaxy. It is calculated based on the total influence of baryonic matter and its distribution within the effective radius.

- **Delta_P_Dark**
  - **Description**: The dark pressure differential ($\Delta P_{\text{dark}}$) represents the pressure difference attributed to dark matter or other non-baryonic influences. This value is derived similarly to the baryonic pressure differential but accounts for the dark influence within the effective radius.

- **k_Baryonic**
  - **Description**: The k value for baryonic influence is a proportionality constant that links the baryonic pressure differential to the total baryonic influence. It provides a scaling factor that helps quantify the impact of baryonic matter within the galaxy.

- **k_Dark**
  - **Description**: The k value for dark influence is a proportionality constant that connects the dark pressure differential to the total dark influence. This constant helps to measure the relative impact of dark matter or other non-baryonic influences within the galaxy.

These fields and their descriptions provide a comprehensive understanding of the derived metrics used in the analysis of galactic rotation curves and the influences of baryonic and dark matter.

### Exploring Relationships Between Derived Fields

In this section, we perform a detailed analysis of the relationships between key derived fields within our dataset. We begin by calculating the correlation matrix to understand how these fields are interrelated, visually represented using a heatmap. This correlation analysis helps us identify strong linear relationships that may exist between variables, offering insights into potential dependencies or redundancies.

Following this, we generate summary statistics for selected fields to capture their central tendencies and dispersion, providing a numerical overview of our data. Finally, we use pairplots to visualize the pairwise relationships between these fields, allowing us to explore possible correlations and patterns in a multi-dimensional space. Together, these analyses will enhance our understanding of how these derived fields interact and contribute to the overall behavior observed in our galactic rotation curve models.

In [ ]:
import seaborn as sns

# Load the results CSV file
file_path = 'galaxy_influence_results.csv'
results_df = pd.read_csv(file_path)

# Filter out non-numeric columns for correlation analysis
numeric_columns = results_df.select_dtypes(include=[np.number]).columns
numeric_df = results_df[numeric_columns]

# Perform a correlation analysis
correlation_matrix = numeric_df.corr()

# Plot the correlation matrix
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Matrix of Derived Fields')
plt.show()

# Summary statistics for the new fields
summary_stats = results_df[['Effective_Radius', 'Delta_P_Baryonic', 'Delta_P_Dark', 'k_Baryonic', 'k_Dark']].describe()

# Pairplot to visualize relationships between new fields
sns.pairplot(results_df[['Effective_Radius', 'Delta_P_Baryonic', 'Delta_P_Dark', 'k_Baryonic', 'k_Dark']])
plt.suptitle('Pairplot of Derived Fields', y=1.02)
plt.show()

summary_stats

# Regression Analysis

### Linear Regression and Clustering Analysis

In this section, we apply linear regression and clustering techniques to further explore the relationships between our key variables. First, we perform linear regression to model the relationships between the derived fields (`Effective_Radius`, `Delta_P_Baryonic`, and `Delta_P_Dark`) and the coefficients `k_Baryonic` and `k_Dark`. This allows us to assess how well these variables explain the variance in `k_Baryonic` and `k_Dark` and identify the most influential factors.

Next, we use K-means clustering to categorize galaxies based on their `Effective_Radius` and pressure differentials (`Delta_P_Baryonic` and `Delta_P_Dark`). The clustering results are evaluated using the silhouette score, which measures how well the clusters are defined. We visualize the clusters to gain insights into distinct groupings within the dataset, highlighting patterns that may not be evident through regression alone.

Finally, the results of the regression analysis, including the coefficients and model scores, along with the clustering outcomes, are presented to provide a comprehensive overview of the underlying relationships and groupings in the data.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

# Prepare data for regression analysis
X = results_df[['Effective_Radius', 'Delta_P_Baryonic', 'Delta_P_Dark']]
y_k_baryonic = results_df['k_Baryonic']
y_k_dark = results_df['k_Dark']

# Perform linear regression for k_Baryonic
reg_k_baryonic = LinearRegression().fit(X, y_k_baryonic)
reg_k_baryonic_score = reg_k_baryonic.score(X, y_k_baryonic)
reg_k_baryonic_coefficients = reg_k_baryonic.coef_

# Perform linear regression for k_Dark
reg_k_dark = LinearRegression().fit(X, y_k_dark)
reg_k_dark_score = reg_k_dark.score(X, y_k_dark)
reg_k_dark_coefficients = reg_k_dark.coef_

# Perform K-means clustering
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X)
silhouette_avg = silhouette_score(X, clusters)

# Add cluster information to the DataFrame
results_df['Cluster'] = clusters

# Plot clusters
plt.figure(figsize=(12, 8))
sns.scatterplot(x='Effective_Radius', y='Delta_P_Baryonic', hue='Cluster', data=results_df, palette='viridis')
plt.title('K-means Clustering of Galaxies Based on Effective Radius and Delta_P_Baryonic')
plt.show()

plt.figure(figsize=(12, 8))
sns.scatterplot(x='Effective_Radius', y='Delta_P_Dark', hue='Cluster', data=results_df, palette='viridis')
plt.title('K-means Clustering of Galaxies Based on Effective Radius and Delta_P_Dark')
plt.show()

# Display regression results and silhouette score
regression_results = {
    'k_Baryonic': {
        'Score': reg_k_baryonic_score,
        'Coefficients': reg_k_baryonic_coefficients
    },
    'k_Dark': {
        'Score': reg_k_dark_score,
        'Coefficients': reg_k_dark_coefficients
    },
    'K-means Clustering': {
        'Silhouette Score': silhouette_avg,
        'Cluster Centers': kmeans.cluster_centers_
    }
}

regression_results

# Fully Implemented Wave Equations for Predictions

### Merging SPARC Dataset with Influence Data

In this section, we merge the SPARC dataset with the previously derived influence data to create a comprehensive dataset for further analysis. The merging is performed on the `galaxy` column, ensuring that corresponding data from both sources are aligned.

We then rename specific columns to maintain consistency in naming conventions and remove any duplicate columns that may have arisen during the merge process. The resulting dataset is saved as `merged_data.csv` for easy access in subsequent analyses.

Finally, we inspect the first few rows of the merged dataset to verify the success of the merge and ensure the integrity of the data.

In [ ]:
# Load the SPARC dataset and influence data
sparc_data = pd.read_csv('sparc_dataset.csv')
influence_df = pd.read_csv('galaxy_influence_results.csv')

# Merge the data on the galaxy column
merged_data = pd.merge(sparc_data, influence_df, on='galaxy', how='left')

# Rename the columns appropriately
merged_data.rename(columns={
    'radius_x': 'radius',
    'Vobs_x': 'Vobs',
    'errV_x': 'errV',
    'Vgas_x': 'Vgas',
    'Vdisk_x': 'Vdisk',
    'Vbul_x': 'Vbul',
    'SBdisk_x': 'SBdisk',
    'SBbul_x': 'SBbul'
}, inplace=True)

# Drop the duplicate columns with suffix _y if they exist
merged_data = merged_data.loc[:, ~merged_data.columns.str.endswith('_y')]
merged_data.to_csv('merged_data.csv', index=False)

# Inspect the first few rows of each DataFrame
# print(sparc_data.head())
# print(influence_df.head())

print(merged_data.head())

# Plotting Predicting Velocity vs. Wave Equation Solution

### Calculation and Analysis of Galaxy Rotation Curves Using Wave Equations

In this section, we perform a comprehensive analysis of galaxy rotation curves using wave equations and various gravitational influence metrics. The process is broken down into several key steps:

1. **Constants and Conversion Factors:** We define essential constants, including the gravitational constant $G$, conversion factors, and assume a constant $w_z$ length for simplicity.

2. **Mass and Radius Calculations:** Functions are defined to calculate the total gravitational influence based on the radius and observed velocity of a galaxy, and to compute the effective radius by incorporating the spatial radius and $w_z$ component.

3. **Potential Function Analysis:** We develop functions to analyze the potential function $V(w)$, calculating key statistics such as mean, median, and standard deviation, along with performing a linear regression analysis to understand the trend of $V(w)$ across the dataset.

4. **Numerical Solver for Wave Equations:** We implement a numerical solver for the wave equation that models the gravitational influence using the derived potential function. This solution is then used to predict the rotation curves of galaxies.

5. **Influence Decomposition:** We decompose the total gravitational influence into baryonic and dark matter components using the dynamic and static Nye factors, respectively. Pressure differentials are calculated to determine the coefficients **k_baryonic** and **k_dark**.

6. **Galaxy-Specific Analysis:** For each galaxy, we perform the above calculations, analyze the potential function, solve the wave equation, and predict the rotational velocity curve. The results are plotted and saved for each galaxy, providing a visual comparison between observed and predicted velocities.

7. **Results Compilation:** The results for all galaxies are compiled into a single DataFrame and exported to a CSV file for further analysis. This includes the predicted rotational velocities and other calculated metrics.

This detailed analysis allows us to model and predict galaxy rotation curves more accurately, integrating the theoretical framework with observational data.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os

# Constants
G = 6.67430e-11  # Gravitational constant in m^3 kg^-1 s^-2
kpc_to_m = 3.086e+19  # Conversion factor from kpc to meters
WZ = 1.0  # Assuming a constant WZ length for simplicity
c = 3.0e8  # Speed of light in m/s


def calculate_total_influence(radius, velocity):
    """
    Calculate the total mass at a given radius point based on the observed velocity.
    Parameters:
    radius (float): Radius in kiloparsecs (kpc).
    velocity (float): Observed velocity in kilometers per second (km/s).
    Returns:
    float: Total mass influence in kg.
    """
    radius_m = radius * kpc_to_m  # Convert radius from kpc to meters
    velocity_m_s = velocity * 1000  # Convert velocity from km/s to m/s
    total_influence = (radius_m * velocity_m_s**2) / G
    return total_influence

def calculate_effective_radius(radius):
    """
    Calculate the effective radius given the spatial radius and WZ component.
    Parameters:
    radius (float): Spatial radius in kiloparsecs (kpc).
    Returns:
    float: Effective radius.
    """
    return np.sqrt(radius**2 + WZ**2)


# Functions for analysis and calculations
def analyze_potential_function(V_w):
    mean_V = np.mean(V_w)
    median_V = np.median(V_w)
    std_V = np.std(V_w)
    slope, intercept, r_value, p_value, std_err = linregress(range(len(V_w)), V_w)
    return mean_V, median_V, std_V, slope, intercept, r_value

def plot_potential_function(V_w):
    plt.figure(figsize=(10, 6))
    plt.plot(V_w, label='Potential Function V(w)')
    plt.xlabel('Index')
    plt.ylabel('Potential Function V(w)')
    plt.title('Potential Function V(w) Analysis')
    plt.legend()
    plt.grid(True)
    plt.show()

def calculate_potential_function(galaxy_data):
    k_baryonic = galaxy_data['k_Baryonic'].mean()
    V_w = k_baryonic * galaxy_data['radius']
    return V_w

def numerical_solver_wave_equation(V_w, w_max, num_points):
    w = np.linspace(0, w_max, num_points)
    delta_w = w[1] - w[0]
    W_w = np.zeros(num_points)
    W_w[0] = V_w.iloc[0]  # Initial value from the potential function
    W_w[1] = V_w.iloc[1]
    for i in range(2, num_points):
        W_w[i] = (2 - 2 * delta_w**2 * V_w.iloc[i-1]) * W_w[i-1] - W_w[i-2]
    return W_w

def calculate_predicted_velocity_wave(galaxy_data, W_w_solution):
    r = galaxy_data['radius'].values * kpc_to_m
    total_mass = galaxy_data['Total_Influence'].values

    # Use the wave function solution W_w to influence the gravitational calculations
    influence = np.abs(W_w_solution)

    # Calculate baryonic and dark components using the wave function influence
    V_baryonic = np.sqrt((G * total_mass * influence) / r)
    V_dark = np.sqrt((G * total_mass * (1 - influence)) / r)

    # Combine the components to get the predicted velocity
    Vpred = np.sqrt(V_baryonic**2 + V_dark**2)

    return r / kpc_to_m, Vpred / 1000  # Convert back to kpc and km/s

# Function to decompose influence components
def decompose_influence_components_wave(galaxy_data):
    # Calculate Total Influence
    # Ensure the correct columns are being accessed
    if 'Total_Influence' not in galaxy_data.columns:
        print("Total_Influence column is missing")
        return galaxy_data

    if 'radius' not in galaxy_data.columns or 'Vobs' not in galaxy_data.columns:
        print("Necessary columns are missing: radius or Vobs")
        return galaxy_data

    galaxy_data['Total_Influence'] = galaxy_data.apply(lambda row: calculate_total_influence(row['radius'], row['Vobs']), axis=1)

    # Calculate Effective Radius
    galaxy_data['Effective_Radius'] = galaxy_data['radius'].apply(calculate_effective_radius)

    # Calculate Baryonic Influence using the dynamic Nye factor
    galaxy_data['Baryonic_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['Nye_Factor_Dynamic']

    # Calculate Dark Influence using the static Nye factor
    galaxy_data['Dark_Influence'] = galaxy_data['Total_Influence'] * galaxy_data['Nye_Factor_Static']

    # Calculate pressure differentials
    galaxy_data['Delta_P_Baryonic'] = galaxy_data.apply(lambda row: (row['Total_Influence'] * row['radius']) / (row['Effective_Radius']**2), axis=1)
    galaxy_data['Delta_P_Dark'] = galaxy_data.apply(lambda row: (row['Total_Influence'] * WZ) / (row['Effective_Radius']**2), axis=1)

    # Solve for k
    galaxy_data['k_Baryonic'] = galaxy_data.apply(lambda row: row['Baryonic_Influence'] / (row['Delta_P_Baryonic'] * row['Effective_Radius']), axis=1)
    galaxy_data['k_Dark'] = galaxy_data.apply(lambda row: row['Dark_Influence'] / (row['Delta_P_Dark'] * row['Effective_Radius']), axis=1)

    return galaxy_data

# Initialize a DataFrame to store results for all galaxies
results_df = pd.DataFrame()

# Plot the observed vs. predicted rotation curves for each galaxy and save them
unique_galaxies = merged_data['galaxy'].unique()

# Create a directory to save the plots
plots_dir = 'galaxy_plots_wave'
os.makedirs(plots_dir, exist_ok=True)

for galaxy in unique_galaxies:
    galaxy_data = merged_data[merged_data['galaxy'] == galaxy].copy()
    galaxy_data = decompose_influence_components_wave(galaxy_data)

    # Calculate the potential function for the entire dataset
    V_w_analysis = calculate_potential_function(galaxy_data)

    # Analyze the potential function
    mean_V, median_V, std_V, slope, intercept, r_value = analyze_potential_function(V_w_analysis)
    print(f'Galaxy: {galaxy}')
    print(f'Mean: {mean_V}, Median: {median_V}, Std Dev: {std_V}')
    print(f'Linear Regression Slope: {slope}, Intercept: {intercept}, R-squared: {r_value**2}')

    # Plot the potential function
    plot_potential_function(V_w_analysis)

    # Numerical solution of the wave equation with the linear boundary condition
    w_max = np.max(galaxy_data['radius'])
    num_points = len(galaxy_data)
    W_w_solution = numerical_solver_wave_equation(V_w_analysis, w_max, num_points)

    # Predict rotation curve using the wave function solution
    radii, Vpred = calculate_predicted_velocity_wave(galaxy_data, W_w_solution)

    # Append additional data to the galaxy_data DataFrame
    galaxy_data['Predicted_Velocity'] = Vpred

    # Print the galaxy data for verification
    print(galaxy_data)

    # Append galaxy data to the results_df DataFrame
    results_df = pd.concat([results_df, galaxy_data], ignore_index=True)

    # Plot the rotation curve and W_w solution
    fig, ax1 = plt.subplots()

    ax1.errorbar(galaxy_data['radius'], galaxy_data['Vobs'], yerr=galaxy_data['errV'], fmt='o', label='Observed')
    ax1.plot(radii, Vpred, label='Predicted', linestyle='--')
    ax1.set_xlabel('Radius (kpc)')
    ax1.set_ylabel('Velocity (km/s)')
    ax1.set_title(f'Rotation Curve for Galaxy: {galaxy}')
    ax1.legend(loc='upper left')

    # Create a second y-axis for the W_w_solution
    ax2 = ax1.twinx()
    ax2.plot(np.linspace(0, w_max, num_points), W_w_solution, label='Wave Eq. Solution', color='r', alpha=0.7, lw=2.25)
    ax2.set_ylabel('Wave Eq. Solution')
    ax2.legend(loc='upper right')

    fig.tight_layout()  # Adjust layout to prevent overlap

    # Save the plot
    plot_filename = os.path.join(plots_dir, f'{galaxy}_rotation_curve_new.png')
    plt.savefig(plot_filename)

    plt.show()

# Export the results DataFrame to a CSV file
results_csv_path = 'galaxy_influence_results_predicted_full.csv'
results_df.to_csv(results_csv_path, index=False)

results_df.head()


## Error metrics

### Error Metrics and Performance Evaluation of the Wave Equation Method

In this section, we implement functions to assess the accuracy of the wave equation method in predicting galaxy rotation curves. The evaluation process involves the following steps:

1. **Error Metrics Calculation:** We define a function `calculate_error_metrics` that computes the Mean Squared Error (MSE) and Root Mean Squared Error (RMSE) between the observed and predicted velocities for each galaxy. These metrics provide a quantitative measure of the model's performance, with lower values indicating better accuracy.

2. **Performance Evaluation:** The `evaluate_performance` function iterates through all galaxies in the dataset, calculating the MSE and RMSE for each. It stores these metrics, allowing us to rank the galaxies based on their RMSE scores.

3. **Identifying Best and Worst Performers:** After calculating the error metrics, galaxies are sorted by their RMSE scores. The three best-performing galaxies (with the lowest RMSE) and the three worst-performing galaxies (with the highest RMSE) are identified. This helps in understanding where the model excels and where it may need improvement.

4. **Overall Error Metrics:** We calculate the overall MSE and RMSE across all galaxies to provide a general sense of the model's accuracy.

5. **Results Output:** The overall error metrics, along with the details of the best and worst-performing galaxies, are printed for further analysis. This information is crucial for evaluating the effectiveness of the wave equation method and identifying potential areas for refinement.

This analysis allows us to systematically evaluate and compare the model's predictive accuracy across different galaxies, offering insights into its strengths and limitations.

In [ ]:
# Function to calculate MSE and RMSE
def calculate_error_metrics(observed, predicted):
    mse = mean_squared_error(observed, predicted)
    rmse = np.sqrt(mse)
    return mse, rmse

# Function to evaluate the performance of the wave equation method
def evaluate_performance(results_df):
    performance_data = []

    # Calculate errors for each galaxy and store the results
    galaxies = results_df['galaxy'].unique()
    for galaxy in galaxies:
        galaxy_data = results_df[results_df['galaxy'] == galaxy]
        mse, rmse = calculate_error_metrics(galaxy_data['Vobs'], galaxy_data['Predicted_Velocity'])
        performance_data.append((galaxy, mse, rmse))

    # Sort galaxies based on their RMSE
    sorted_galaxies = sorted(performance_data, key=lambda x: x[2])

    # Select three best and three worst performing galaxies
    best_galaxies = sorted_galaxies[:3]
    worst_galaxies = sorted_galaxies[-3:]

    return performance_data, best_galaxies, worst_galaxies

print(results_df.head())

# Assuming results_df contains the predicted and observed velocities for all galaxies
performance_data, best_galaxies, worst_galaxies = evaluate_performance(results_df)

# Overall MSE and RMSE
overall_mse = np.mean([data[1] for data in performance_data])
overall_rmse = np.mean([data[2] for data in performance_data])

print(f"Overall MSE: {overall_mse}")
print(f"Overall RMSE: {overall_rmse}")

print("\nThree Best Performing Galaxies:")
for galaxy, mse, rmse in best_galaxies:
    print(f"Galaxy: {galaxy}, MSE: {mse}, RMSE: {rmse}")

print("\nThree Worst Performing Galaxies:")
for galaxy, mse, rmse in worst_galaxies:
    print(f"Galaxy: {galaxy}, MSE: {mse}, RMSE: {rmse}")

# Statistical Analysis

### Data Range, Type Checks, and Statistical Analysis of Predicted vs. Observed Velocities

This section is focused on ensuring the integrity of the data and performing a comprehensive statistical analysis to evaluate the accuracy of the wave equation model in predicting galaxy rotation curves. The process includes the following steps:

1. **Data Range and Type Checks:**  
   The `check_data` function is used to verify the range and data types of the observed and predicted velocity values for each galaxy. By sampling the first few values and checking their data types and ranges, we ensure that the data is formatted correctly and falls within expected limits. This step is crucial for identifying any anomalies or data type mismatches early in the analysis.

2. **Statistical Analysis:**  
   The `statistical_analysis` function computes various statistical metrics to assess the model's performance:
   - **Mean Squared Error (MSE)** and **Root Mean Squared Error (RMSE)** quantify the average squared and root-squared differences between observed and predicted velocities.
   - **Mean Absolute Error (MAE)** provides the average absolute difference between observed and predicted values.
   - **R-squared** and **Adjusted R-squared** indicate the proportion of variance in the observed data that is explained by the model's predictions.
   - **Bias** measures the average difference between predicted and observed values, providing insight into any systematic over- or under-prediction.

3. **Per-Galaxy Analysis:**  
   For each galaxy, the model's predictions are compared against observed data, and the statistical metrics are calculated. The results are printed, allowing for detailed evaluation of the model's performance for each individual galaxy.

4. **Output and Verification:**  
   After performing the statistical analysis, the results are printed for each galaxy, providing a clear and concise summary of how well the model's predictions align with the observed rotation curves. Additionally, the data range and type checks ensure that the data used in these calculations is valid and reliable.

This section ensures both the accuracy of the data and the effectiveness of the wave equation model through rigorous statistical evaluation, providing confidence in the model's predictions and identifying areas for further refinement​⬤

In [ ]:
# Check Data Range and Types Function
def check_data(galaxy_data):
    observed = galaxy_data['Vobs'].values
    predicted = galaxy_data['Predicted_Velocity'].values

    print("Sample of Observed Values:", observed[:5])
    print("Sample of Predicted Values:", predicted[:5])
    print("Data Type of Observed Values:", observed.dtype)
    print("Data Type of Predicted Values:", predicted.dtype)
    print("Range of Observed Values: Min =", np.min(observed), ", Max =", np.max(observed))
    print("Range of Predicted Values: Min =", np.min(predicted), ", Max =", np.max(predicted))

# Statistical Analysis Function
def statistical_analysis(galaxy_data):
    observed = galaxy_data['Vobs'].values
    predicted = galaxy_data['Predicted_Velocity'].values

    # Ensure that observed and predicted are floats
    observed = observed.astype(float)
    predicted = predicted.astype(float)

    # Calculate Mean Squared Error (MSE)
    mse = np.mean((observed - predicted) ** 2)

    # Calculate Root Mean Squared Error (RMSE)
    rmse = np.sqrt(mse)

    # Calculate Mean Absolute Error (MAE)
    mae = np.mean(np.abs(observed - predicted))

    # Calculate R-squared
    ss_res = np.sum((observed - predicted) ** 2)
    ss_tot = np.sum((observed - np.mean(observed)) ** 2)
    r_squared = 1 - (ss_res / ss_tot)

    # Calculate Adjusted R-squared
    n = len(observed)
    p = 1  # Number of predictors (since we are using the model's predictions)
    adj_r_squared = 1 - ((1 - r_squared) * (n - 1) / (n - p - 1))

    # Calculate Bias
    bias = np.mean(predicted - observed)

    return {
        'MSE': mse,
        'RMSE': rmse,
        'MAE': mae,
        'R-squared': r_squared,
        'Adjusted R-squared': adj_r_squared,
        'Bias': bias
    }

# Perform statistical analysis for each galaxy
for galaxy in unique_galaxies:
    galaxy_data = merged_data[merged_data['galaxy'] == galaxy].copy()
    galaxy_data = decompose_influence_components_wave(galaxy_data)

    # Calculate the potential function for the entire dataset
    V_w_analysis = calculate_potential_function(galaxy_data)

    # Numerical solution of the wave equation with the linear boundary condition
    w_max = np.max(galaxy_data['radius'])
    num_points = len(galaxy_data)
    W_w_solution = numerical_solver_wave_equation(V_w_analysis, w_max, num_points)

    # Predict rotation curve using the wave function solution
    radii, Vpred = calculate_predicted_velocity_wave(galaxy_data, W_w_solution)

    # Append predicted velocities to the galaxy_data DataFrame
    galaxy_data['Predicted_Velocity'] = Vpred

    # Perform statistical analysis
    stats = statistical_analysis(galaxy_data)

    print(f'\nStatistical Analysis for Galaxy: {galaxy}')
    for key, value in stats.items():
        print(f'{key}: {value:.6f}')

    # Check data range and types
    check_data(galaxy_data)

    print("\n")

# Sensitivity Analysis

### Wave Equations

### Sensitivity Analysis of Galaxy Rotation Curves: Understanding the Dominance of Radius $r$

This section focuses on performing a sensitivity analysis to investigate how different parameters, specifically \(k_Baryonic\), \(Nye_Factor_Static\), and \(Delta_P_Baryonic\), affect the predicted velocities and related statistics of galaxy rotation curves. The goal is to track changes in the predicted velocities and the mean and standard deviation of \(k_Baryonic\) as we vary these parameters.

#### Key Steps and Insights:

1. **Sensitivity Analysis Function:**
   - We define the `sensitivity_analysis_track_k_baryonic` function to systematically vary a single parameter (e.g., **k_Baryonic**, **Nye_Factor_Static**, **Delta_P_Baryonic**) and observe its impact on the predicted galaxy velocities.
   - For each parameter value, we decompose the influence components, solve the wave equation, and calculate the predicted velocities. We also track the mean and standard deviation of **k_Baryonic** to see how it changes with different parameter values.

2. **Parameter Variation and Result Tracking:**
   - The analysis is conducted for several galaxies, with a range of values for each parameter to cover a broad spectrum of possible scenarios.
   - The results are stored in DataFrames, which include both the predicted velocities and the changes in **k_Baryonic** for each parameter value.

3. **Dominance of Radius $r$:**
   - Throughout the analysis, we found that modifying parameters like **k_Baryonic**, **Nye_Factor_Static**, and **Delta_P_Baryonic** did not significantly affect the predicted outcomes. The lack of variance in results when tweaking these parameters revealed that the radius $r$ dominates the behavior of the model.
   - This insight suggests that the wave equation's prediction of galaxy rotation curves is heavily dependent on the radius, and other parameters have minimal impact on the final results. This dominance of $r$ underscores its critical role in the model, indicating that the gravitational effects are primarily governed by spatial dimensions rather than the specific values of these other parameters.

4. **Output and Conclusion:**
   - The results, including sensitivity statistics and tracking of **k_Baryonic** changes, are exported to CSV files for further analysis.
   - The findings provide valuable insights into the robustness of the model, showing that while various parameters can be adjusted, the radius $r$ remains the key determinant in the predicted rotation curves, highlighting the model's sensitivity to spatial factors over other influence components.

This analysis helps refine our understanding of the model's behavior and confirms the central role of the radius $r$ in determining galaxy rotation curves, with other parameters playing a much lesser role.

In [ ]:
import pandas as pd
import numpy as np

# Function to perform sensitivity analysis and track k_Baryonic changes
def sensitivity_analysis_track_k_baryonic(galaxy_data, param_name, param_values):
    """
    Perform sensitivity analysis by varying a single parameter, observing its effect on the predicted velocities,
    and tracking changes in the mean and standard deviation of k_Baryonic.

    Parameters:
    galaxy_data (pd.DataFrame): DataFrame containing the galaxy data.
    param_name (str): The name of the parameter to vary.
    param_values (list): List of values for the parameter.

    Returns:
    pd.DataFrame: Summary DataFrame aggregating statistics of predicted velocity by the parameter.
    pd.DataFrame: DataFrame tracking k_Baryonic statistics (mean and std) for each parameter value.
    """
    results = []
    k_baryonic_tracking = []

    original_values = galaxy_data.copy()

    for param_value in param_values:
        # Set the parameter value
        galaxy_data[param_name] = param_value

        # Decompose influence components with the updated parameter
        galaxy_data = decompose_influence_components_wave(galaxy_data)

        # Predict rotation curve using the existing process
        V_w_analysis = calculate_potential_function(galaxy_data)
        w_max = np.max(galaxy_data['radius'])
        num_points = len(galaxy_data)
        W_w_solution = numerical_solver_wave_equation(V_w_analysis, w_max, num_points)
        radii, Vpred = calculate_predicted_velocity_wave(galaxy_data, W_w_solution)

        # Calculate statistics for the predicted velocities
        results.append({
            param_name: param_value,
            'Mean_Predicted_Velocity': np.mean(Vpred),
            'Std_Predicted_Velocity': np.std(Vpred),
            'Min_Predicted_Velocity': np.min(Vpred),
            'Max_Predicted_Velocity': np.max(Vpred)
        })

        # Track the mean and standard deviation of k_Baryonic
        k_baryonic_mean = galaxy_data['k_Baryonic'].mean()
        k_baryonic_std = galaxy_data['k_Baryonic'].std()
        k_baryonic_tracking.append({
            param_name: param_value,
            'Mean_k_Baryonic': k_baryonic_mean,
            'Std_k_Baryonic': k_baryonic_std
        })

        # Reset galaxy_data to original values for next iteration
        galaxy_data = original_values.copy()

    return pd.DataFrame(results), pd.DataFrame(k_baryonic_tracking)

# Define parameter values for sensitivity analysis
static_values = [1e-10, 0.25, 0.5, 0.75, 1, 5, 10, 15]   # Different values of Nye_Factor_Static
k_b_values = [1e-10, 0.25, 0.5, 0.75, 1, 5, 10, 15]      # Different values of k_Baryonic
delta_p_values = [1e-50, 1e-25, 1e-10, 1, 1e10, 1e24, 1e50]  # Different values of Delta_P_Baryonic

# Select multiple galaxies for the analysis
# galaxies_to_analyze = ['UGC12506', 'NGC2403', 'IC2574']
galaxies_to_analyze = unique_galaxies

# Initialize DataFrames to store results
all_k_baryonic_stats = pd.DataFrame()
all_k_baryonic_tracking = pd.DataFrame()

# Run the sensitivity analysis for each selected galaxy and each parameter separately
for galaxy in galaxies_to_analyze:
    print(f"Analyzing galaxy: {galaxy}")
    sample_galaxy_data = merged_data[merged_data['galaxy'] == galaxy].copy()

    # Run the sensitivity analysis for k_Baryonic
    k_baryonic_stats, k_baryonic_tracking = sensitivity_analysis_track_k_baryonic(sample_galaxy_data, 'k_Baryonic', k_b_values)

    # Add galaxy information to tracking DataFrame
    k_baryonic_tracking['Galaxy'] = galaxy
    k_baryonic_stats['Galaxy'] = galaxy

    # Append the results for this galaxy to the overall DataFrame
    all_k_baryonic_stats = pd.concat([all_k_baryonic_stats, k_baryonic_stats], ignore_index=True)
    all_k_baryonic_tracking = pd.concat([all_k_baryonic_tracking, k_baryonic_tracking], ignore_index=True)

# Export the results to CSV files
all_k_baryonic_stats.to_csv('All_K_Baryonic_Sensitivity_Stats.csv', index=False)
all_k_baryonic_tracking.to_csv('All_K_Baryonic_Tracking.csv', index=False)

# Print the summary tables
print("k_Baryonic Sensitivity Statistics:")
print(all_k_baryonic_stats)

print("\nTracking changes in k_Baryonic (mean and std):")
print(all_k_baryonic_tracking)

# Best and Worst Plot Generation

### Trigonometric Method

### Visualization of Observed vs. Predicted Rotation Curves for Best and Worst Performing Galaxies

This section focuses on visualizing the comparison between observed and predicted galaxy rotation curves, specifically highlighting the best and worst performing galaxies based on their R-squared values. The process is as follows:

1. **Loading and Preparing Data:**
   - The results are loaded from a CSV file, where the observed (`Vobs`) and predicted (`Vpred`) velocity data are stored as string representations of lists. These strings are converted back into numpy arrays for further analysis.

2. **Calculating R-squared Values:**
   - The R-squared values, which indicate the proportion of variance explained by the model, are calculated for each galaxy by comparing the observed and predicted velocities. These values help determine the accuracy of the model's predictions.

3. **Selecting Best and Worst Performers:**
   - The galaxies are ranked based on their R-squared values, and the four best and four worst performing galaxies are selected. This selection allows for a focused analysis of where the model performs well and where it may struggle.

4. **Plotting Rotation Curves:**
   - A 2x4 grid of subplots is created to visualize the rotation curves of the selected galaxies. Each subplot includes:
     - An error bar plot comparing the observed and predicted velocities across the galaxy's radius.
     - A secondary plot on the same graph that shows the percentage of baryonic and dark matter influence as a function of radius.

5. **Dual-Axis Representation:**
   - The dual-axis plot effectively displays the relationship between observed/predicted velocities and the corresponding influence of baryonic and dark matter components. This allows for a comprehensive view of how well the model captures the dynamics of different galaxies.

6. **Layout and Display:**
   - The layout is adjusted to ensure that all plots are clearly visible without overlap, and the entire figure is displayed to provide a visual comparison across the best and worst performing galaxies.

This visualization is crucial for understanding the strengths and weaknesses of the model by directly comparing its predictions with observational data across galaxies with varying levels of performance.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Load the CSV file with results
results_file_path = 'trigonometric_results.csv'
df_results = pd.read_csv(results_file_path)

# Function to convert string representation of a list back to a numpy array
def convert_to_array(column):
    return np.array(eval(column))

# Convert the Vobs and Vpred columns from string to numpy arrays
df_results['Vobs'] = df_results['Vobs'].apply(convert_to_array)
df_results['Vpred'] = df_results['Vpred'].apply(convert_to_array)

# Determine the R-squared values for each galaxy's observed vs. predicted velocities
df_results['R_squared'] = df_results.apply(lambda row: np.corrcoef(row['Vobs'], row['Vpred'])[0, 1]**2, axis=1)

# Find the four best and four worst performing galaxies based on R-squared values
best_galaxies = df_results.nlargest(4, 'R_squared')
worst_galaxies = df_results.nsmallest(4, 'R_squared')

# Combine the best and worst galaxies into one DataFrame
selected_galaxies = pd.concat([best_galaxies, worst_galaxies])

# Create a plot with eight subplots in a 2x4 grid
fig, axes = plt.subplots(nrows=2, ncols=4, figsize=(20, 10))
axes = axes.flatten()

# Plot the observed vs. predicted rotation curves and influence percentages
for i, (index, galaxy_data) in enumerate(selected_galaxies.iterrows()):
    ax1 = axes[i]

    # Convert string representation of lists back to lists
    radius = np.fromstring(galaxy_data['radius'][1:-1], sep=',')
    Vobs = galaxy_data['Vobs']
    Vpred = galaxy_data['Vpred']
    percent_baryonic = np.fromstring(galaxy_data['Percent_Baryonic'][1:-1], sep=',')
    percent_dark = np.fromstring(galaxy_data['Percent_Dark'][1:-1], sep=',')

    # Plot the observed and predicted rotation curves
    ax1.errorbar(radius, Vobs, fmt='o', label='Observed', alpha=0.7)
    ax1.plot(radius, Vpred, label='Predicted', linestyle='--')
    ax1.set_xlabel('Radius (kpc)')
    ax1.set_ylabel('Velocity (km/s)')
    ax1.set_title(f'Galaxy: {galaxy_data["galaxy"]}')
    ax1.legend(loc='upper left')

    # Create a second y-axis for the percentage of baryonic and dark influence
    ax2 = ax1.twinx()
    ax2.plot(radius, percent_baryonic, label='Baryonic Influence %', color='g', alpha=0.5)
    ax2.plot(radius, percent_dark, label='Dark Influence %', color='purple', alpha=0.5)
    ax2.set_ylabel('Percentage')
    ax2.legend(loc='upper right')

fig.tight_layout()  # Adjust layout to prevent overlap
plt.show()